In [6]:
import requests
import pandas as pd
from collections import defaultdict
import numpy as np
import warnings
from datetime import datetime, timedelta
warnings.filterwarnings("ignore")

In [10]:
!pip install alpaca-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 9.0 MB/s eta 0:00:00


In [3]:
def trade_clean(quotes):
  keys = list(quotes.keys())
  for key in keys:
    for i in range(len(quotes[key])):
      cur_dict = quotes[key][i]
      trade_dict['lastPrice'].append(cur_dict['p'])
      trade_dict['Volume'].append(cur_dict['s'])
      trade_dict['ClockTime'].append(cur_dict['t'])

In [18]:
# Your API credentials
headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": "AKUIJAC2GKCCEFQDKAYM",
    "APCA-API-SECRET-KEY": "UYjfIbkSIQr65ImyACdZzrZg728dWspwBSJM2biq"
}

# List of dates for which you want to fetch data
dates = ["2022-01-03"]  # Add your dates here

# Function to format datetime for URL
def format_datetime_for_url(dt):
    return dt.strftime("%Y-%m-%dT%H:%M:%SZ")


# Loop over each date
for date in dates:
  trade_dict = defaultdict(list)
  # Define time ranges
  time_ranges = [("09:30:00", "12:00:00"), ("13:00:00", "15:00:00"),("15:00:00","16:00:00")]

  # Loop over each time range
  for start_time, end_time in time_ranges:
      start_datetime = datetime.fromisoformat(f"{date}T{start_time}")
      end_datetime = datetime.fromisoformat(f"{date}T{end_time}")

      while start_datetime < end_datetime:
          # Calculate the end time for the current one-minute window
          current_end_datetime = start_datetime + timedelta(minutes=1)

          # Ensure the end time does not exceed the overall end time
          if current_end_datetime > end_datetime:
              current_end_datetime = end_datetime

          # Construct the URL for the current one-minute window
          url_start = format_datetime_for_url(start_datetime)
          url_end = format_datetime_for_url(current_end_datetime)
          url = f"https://data.alpaca.markets/v2/stocks/trades?symbols=AAPL&start={url_start}&end={url_end}&limit=10000&feed=sip&sort=asc"

          # Make the request
          response = requests.get(url, headers=headers)

          # Check if the request was successful
          if response.status_code == 200:
              # Process the response
              data = response.json()
              trades = data.get('trades', {})
              trade_clean(trades)
          # Move to the next one-minute window
          start_datetime = current_end_datetime
  trade_df = pd.DataFrame.from_dict(trade_dict)
  trade_df['ClockTime'] = pd.to_datetime(trade_df['ClockTime'], errors='coerce')
  trade_df['ClockTime'] = trade_df['ClockTime'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')
  twap_df = trade_df.groupby('ClockTime').apply(
    lambda x: (x['lastPrice'] * x['Volume']).sum() / x['Volume'].sum()
).reset_index(name='TWAP')
  trade_df = pd.merge(trade_df, twap_df, on='ClockTime', how='left')
  trade_df = trade_df.drop_duplicates(subset=['ClockTime'])
  trade_df = trade_df.drop('lastPrice',axis=1)

In [22]:
trade_df

,Volume,ClockTime,TWAP
0,1,2022-01-03 09:30:01.929000,177.880
1,56,2022-01-03 09:30:02.394000,177.870
2,100,2022-01-03 09:30:05.946000,177.870
3,7,2022-01-03 09:30:08.832000,177.870
4,100,2022-01-03 09:30:17.763000,177.870
...,...,...,...
287995,15,2022-01-03 15:59:59.276000,180.955
287996,1,2022-01-03 15:59:59.719000,180.950
287997,1,2022-01-03 15:59:59.848000,180.945
287998,1,2022-01-03 15:59:59.874000,180.950
